# Request and Download OOI Data

This notebook creates requests for data and QARTOD QC test results that are available from OOINet and from the OOI dev1 server. QC tests associated with datasets from OOINet have already been implemented in production by the Data Team. The dev1 server is where datasets with results of QARTOD tests in development are hosted. Access to dev1 is restricted to OOI personnel on the internal network.

The requests built below include the retrieval method, data stream, and either the reference designator or site, node, and sensor combination for a specific instrument to request data through the OOI M2M API. The requested datasets can also be limited to a time period defined by start datetime and end datetime parameters.

After downloading the datasets and performing preprocessing to prepare the data for analysis, the datasets are saved locally to an interim data folder for the next step in testing and analyzing QARTOD test results.

### Import modules used in this notebook

In [2]:
# Import libraries available from main conda channels or conda-forge

import os
import re
import netrc
import requests
import gc
import io
import ast
import warnings
warnings.filterwarnings("ignore")
import sys
from tqdm import tqdm

import pandas as pd
import numpy as np
import xarray as xr
import netCDF4
import dask
from dask.diagnostics import ProgressBar

### Using OOINet and ooi-data-explorations libraries locally
To use this notebook as-is, users should fork the OOINet and ooi-data-explorations GitHub repos and clone these to their local GitHub directory. The README for ooi-data-explorations has a detailed description for installing the python library as a local development package [here](https://github.com/oceanobservatories/ooi-data-explorations/tree/master/python#obtaining-the-code-and-configuring-the-environment). These instructions will add the path to ooi-data-explorations to the system path in the resulting ooi environment. Since we use both the OOINet and ooi-data-explorations libraries in this notebook, I like to amend the last step to add both libraries to the path of my base environment. 

Instead of:

    # configure the OOI python environment
    conda env create -f environment.yml
    conda init # might be required for windows users if environment is not active
    conda activate ooi

    # you can check the active environment by running
    conda env list

    # install the package as a local development package
    conda develop .

I would do:

    # Check that ooi environment was successfully created
    conda activate ooi
    conda env list

    # install the package as a local development package in ooi environment
    conda develop .
    
    # Exit the ooi environment
    conda deactivate
    conda env list      # no environments should have an "*" next to it at this point

    # install the package as a local development package in base environment
    conda develop .

Where `.` denotes the current working directory, so users will want to make sure they are still in their local `GitHub/ooi-data-explorations/python/` directory or   `GitHub/OOINet/` directory at this step.

While creating the environment and activating it just to deactivate the environment seems strange, I like having these environments to use later if needed. 

If conda stalls while creating the ooinet environment, that might be caused by libraries in the environment file that are not compatible with your operating system. I start with removing the package build information, and if the problem persists but the error changes slightly then I comment out the problem packages.

In [3]:
# Import OOINet library
from ooinet import M2M
from ooinet.M2M import AUTH 
from ooinet.Instrument.common import process_file

In [4]:
# Import functions from ooi-data-explorations library
from ooi_data_explorations.uncabled.process_ctdbp import ctdbp_datalogger
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations import common as ooi_common
from ooi_data_explorations.common import SESSION

### QARTOD in Production: Request data from the OOINet THREDDS catalog

##### Define data parameters and routines

In [1]:
# Setup parameters needed to request data
refdes = "CP01CNSM-MFD37-03-CTDBPD000"              # Coastal Pioneer Array (NES) - Central Surface Mooring CTD Bottom-pumped, is this the same as site, node, sensor?
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

In [5]:
# Generic preprocessing routine to do some generic dataset cleaning/processing
@dask.delayed
def preprocess(ds):
    ds = xr.open_dataset(ds)
    ds = process_file(ds)
    return ds

##### Using mostly OOINet module

In [6]:
# Use the gold copy THREDDs datasets

thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)

# Get the THREDDs catalog

thredds_catalog = M2M.get_thredds_catalog(thredds_url)
deployments = M2M.get_deployments(refdes)

In [7]:
# Clean the THREDDs catalog

sensor_files = M2M.clean_catalog(thredds_catalog, stream, deployments) 
# sensor_files

# This step separates entries from thredds_catalog if they do not match the stream. These ancillary files are usually provided because they are used in calculating a derived variable from the measured variable stream.

In [8]:
# Now build the url to access the data

sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["goldCopy_dodsC"], file) for file in sensor_files]
# sensor_files

In [9]:
# preprocess the data

zs = [preprocess(file) for file in sensor_files]

In [14]:
# Load all the datasets
# Run this cell twice to download the data if it exits on the first time with a KeyError 
with ProgressBar():
    data = xr.concat([ds.chunk() for ds in dask.compute(*zs)], dim="time")
# data

[########################################] | 100% Completed | 22.70 s


##### Using ooi_data_explorations modules

In [12]:
# Load data with ooi_common module

data = ooi_common.load_gc_thredds(site,node,sensor,method,stream,use_dask=True)    # Request the gold copy data through THREDDs catalog

# It looks like the OOINet module method attempts to avoid collecting ancillary files in addition to the requested sensor files which could add time to the download and open dataset step.
# load_gc_thredds() also calls process_file() within gc_collect() so we achieve the same preprocessing as in the preprocess() defined above.

In [15]:
# Make a copy of the data with a unique name

ds_prod = data.copy()
ds_prod

<xarray.Dataset>
Dimensions:                                            (time: 263600)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) float32 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_pressure                                 (time) float64 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_results   (time) float32 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_pressure_qc_executed                     (time) float32 dask.array<chunksize=(73272,), meta=np.ndarray>
    ctd_time                                           (time) datetime64[ns] dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_practical_salinity_qc_executed           (time) float32 dask.array<chunksize=(73272,), meta=np.ndarray>
    ...                                                 ...
    sea_water_density                                  (time) float64 dask.array<chunksize=(73272,), meta=np.ndarray>
    depth                                              (time) float64 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_pressure_qartod_results                  (time) float32 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_executed  (time) |S64 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_temperature                              (time) float64 dask.array<chunksize=(73272,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed       (time) |S64 dask.array<chunksize=(73272,), meta=np.ndarray>
Attributes: (12/69)
    node:                               MFD37
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13678
    lon:                                -70.76978

In [17]:
interim_data = os.path.relpath('../data/interim')           # path to interim data folder from notebook folder

prod_filename = '-'.join(('prod',ds_prod.id,))+'.nc'        # ds_prod filename from dataset attribute

prod_path=os.path.join(interim_data, prod_filename)         # repeat for ds_prod

ds_prod.to_netcdf(path=prod_path)                           # repeat for ds_prod

### QARTOD in Development: Request data from dev1 server

We may also want to examine new QARTOD tests which are on staging in the Dev-1 environment before they are moved to production. The Development environemt at ooinet-dev1-west.intra.oceanobservatories.org. In order to access data on Dev-1, you need to be granted access and be connected to the CI-West VPN (vpn-west.oceanobservatories.org) at Oregon State.

In [25]:
# Initialize credentials
# This process is borrowed from ooinet.M2M
try:
    nrc = netrc.netrc()
    AUTH = nrc.authenticators('ooinet-dev1-west.intra.oceanobservatories.org')
    login, password = AUTH[0], AUTH[2]
    if AUTH is None:
        raise RuntimeError(
            'No entry found for machine ``ooinet-dev1-west.oceanobservatories.org`` in the .netrc file')
except FileNotFoundError as e:
    raise OSError(e, os.strerror(e.errno), os.path.expanduser('~'))

In [26]:
# Setup parameters needed to request data 
# Check that instrument parameters match an available OOI datasets on dev1 server
# Maybe change this section to look for data sets programatically with ooi-data-explorations functions (list platforms/sites, list methods, list streams,...)

refdes = "CP03ISSM-RID27-03-CTDBPC000"              # Coastal Pioneer Array (NES) - Inshore Surface Mooring Near Surface Instrument Frame - Bottom-pumped CTD
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

The Dev-1 environment has no "goldcopy" equivalent THREDDs catalog. Instead we'll have to do the normal request and wait for the datasets to be assembled and made available for download.

In [27]:
# Sub in ooinet-dev1-west.intra.oceanobservatories.org into the avaialbe API urls

Dev01_urls = {}
for key in M2M.URLS:
    url = M2M.URLS.get(key)
    if "opendap" in url:
        dev1_url = re.sub("opendap", "opendap-dev1-west.intra", url)
    else:
        dev1_url = re.sub("ooinet","ooinet-dev1-west.intra", url)
    Dev01_urls[key] = dev1_url
    
Dev01_urls
   

{'data': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12576/sensor/inv',
 'anno': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12580/anno/find',
 'vocab': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12586/vocab/inv',
 'asset': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12587',
 'deploy': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12587/events/deployment/inv',
 'preload': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12575/parameter',
 'cal': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12587/asset/cal',
 'fileServer': 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/fileServer/',
 'dodsC': 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/',
 'goldCopy': 'https://thredds.dataexplorer.oceanobservatories.org/thredds/catalog/ooigoldcopy/public/',
 'goldCopy_fileServer': 'https://thredds.dataexplorer.oceanobservatories.org/thre

Our choice of URL is similar to the URL used in the M2M example notebook here: https://github.com/ooi-data-review/2018-data-workshops/blob/master/chemistry/examples/quickstart_python.ipynb 
The rest of the data request process through this section is modeled after the linked tutorial above. 

In [28]:
# Use the Dev1 data catalog URL for the request
api_base_url = Dev01_urls['data']

# Use the fileServer URL for downloading data files from the thredds server
tds_url = Dev01_urls['fileServer']

In [29]:

# Create the request URL
data_request_url ='/'.join((api_base_url,site,node,sensor,method,stream))

# We are using a different process for downloading data than in the OOINet section since the default URLs that are set within the other functions connect to OOINet. 
# The development environment also doesn't have a gold copy, although different functions to request non-gold copy datasets from OOINet exist in the OOINet and ooi-data-explorations modules.

# Set optional parameters 
# We specify a date range to control the size of the dataset requested 
params = {
  'beginDT':'2019-09-26T13:50:00.000Z',
  'endDT':'2020-11-01T13:16:00.000Z',
  'format':'application/netcdf',
  'include_provenance':'true',
  'include_annotations':'true'
}

In [30]:
# Build and send the data request
r = requests.get(data_request_url, params=params, auth=(login, password))
dev1_request = r.json()
dev1_request

{'requestUUID': 'ad939421-adeb-4e5c-b106-6967b332718c',
 'outputURL': 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20230510T214036136Z-CP03ISSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html',
 'allURLs': ['https://opendap-dev1-west.intra.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20230510T214036136Z-CP03ISSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html',
  'https://downloads-dev1-west.intra.oceanobservatories.org/async_results/kylene.cooley@whoi.edu/20230510T214036136Z-CP03ISSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered'],
 'sizeCalculation': 44385494,
 'timeCalculation': 60,
 'numberOfSubJobs': 202}

In [31]:
# Download the NetCDF data files from the Dev01 thredds server and load into xarray dataset
url = dev1_request['outputURL']
files = ooi_common.list_files(url)
use_dask = False
frames =[]

for file in tqdm(files, desc='Downloading and Processing Data Files'):
    file_url = re.sub('catalog.html\?dataset=', tds_url, file)
    r = SESSION.get(file_url, timeout=(3.05, 120))
    if r.ok:
        # load the data file
        if use_dask:
            ds = xr.open_dataset(io.BytesIO(r.content), decode_cf=False, chunks=10000)
        else:
            ds = xr.load_dataset(io.BytesIO(r.content), decode_cf=False)

        # Perform preprocessing on opened dataset
        # In this workflow we are omitting preprocessing to the *_qartod_executed variables as in ooi_data_explorations.common.process_file()
        
        ds = ds.swap_dims({'obs': 'time'})              # Swap the primary dimension
        ds = ds.chunk({'time': 100})                    # Used for optimization
        
        ds = ds.reset_coords()
        keys = ['obs', 'id', 'provenance', 'driver_timestamp', 'ingestion_timestamp',
            'port_timestamp', 'preferred_timestamp']
        for key in keys:
            if key in ds.variables:
                ds = ds.drop_vars(key)

        # Since the CF decoding of the time is failing, explicitly reset all instances where the units are
        # seconds since 1900-01-01 to the correct CF units and convert the values to datetime64[ns] types

        time_pattern = re.compile(r'^seconds since 1900-01-01.*$')
        ntp_date = np.datetime64('1900-01-01')
        for v in ds.variables:
            if 'units' in ds[v].attrs.keys():
                if isinstance(ds[v].attrs['units'], str):  # because some units use non-standard characters...
                    if time_pattern.match(ds[v].attrs['units']):
                        del(ds[v].attrs['_FillValue'])  # no fill values for time!
                        ds[v].attrs['units'] = 'seconds since 1900-01-01T00:00:00.000Z'
                        np_time = ntp_date + (ds[v] * 1e9).astype('timedelta64[ns]')
                        ds[v] = np_time

        # Sort by time
        ds = ds.sortby('time') 
        # Clear-up some global attributes we will no longer be using

        keys = ['DODS.strlen', 'DODS.dimName', 'DODS_EXTRA.Unlimited_Dimension', '_NCProperties', 'feature_Type']
        for key in keys:
            if key in ds.attrs:
                del(ds.attrs[key])

        try: 
            ds.encoding['unlimited_dims']
            del ds.encoding['unlimited_dims']
        except KeyError:
            pass

        # Resetting cdm_data_type from Point to Station and the featureType from point to timeSeries

        ds.attrs['cdm_data_type'] = 'Station'
        ds.attrs['featureType'] = 'timeSeries'

        # Update some global attributes

        ds.attrs['acknowledgement'] = 'National Science Foundation'
        ds.attrs['comment'] = 'Data collected from the OOI Dev01 M2M API and reworked for use in locally stored NetCDF files.'
        frames.append(ds)
    else:
        failed_file = file.rpartition('/')
        warnings.warn('Failed to download %s' % failed_file[-1]) 

if frames != []:
    # merge the data frames into a single data set
    data = ooi_common.merge_frames(frames)

In [32]:
# Make a copy of the data with a unique name

ds_dev = data.copy()
ds_dev

<xarray.Dataset>
Dimensions:                                            (time: 736718)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_pressure                                 (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_results   (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_pressure_qc_executed                     (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    ctd_time                                           (time) int32 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_practical_salinity_qc_executed           (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    ...                                                 ...
    sea_water_density                                  (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth                                              (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_pressure_qartod_results                  (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_executed  (time) object dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_temperature                              (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed       (time) object dask.array<chunksize=(10,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI Dev01 M2M...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.367077
    lon:                                -70.881735

### Save datasets to interim data folder for further processing

In [20]:
# We will perform tests on datasets in production and development separately, so each xarray dataset is saved in a separate netCDF file

interim_data = os.path.relpath('../data/interim')           # path to interim data folder from notebook folder

dev_filename = '-'.join(('dev',ds_dev.id,))+'.nc'           # build ds_dev filename from dataset attributes
prod_filename = '-'.join(('prod',ds_prod.id,))+'.nc'        # ds_prod filename from dataset attribute

dev_path=os.path.join(interim_data, dev_filename)           # build full relative path with ds_dev filename
prod_path=os.path.join(interim_data, prod_filename)         # repeat for ds_prod

ds_dev.to_netcdf(path=dev_path)                             # provide both relative path and filename for ds_dev in path parameter
ds_prod.to_netcdf(path=prod_path)                           # repeat for ds_prod